In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
def find_phone_number(name: str) -> str:
    """
    주어진 사람들의 전화번호를 찾아서 반환합니다.
    전화번호를 찾지 못한 경우 None을 반환합니다.
    """
    phone_book = {
        "문벼리": "010-1234-1234",
        "양승렬": "010-2434-2434",
        "장동원": "010-4142-1424",
    }
    if name in phone_book:
        return phone_book[name]
    else:
        return None

In [ ]:
tools = [
    find_phone_number
]

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(
    model_name="gpt-4o-mini"
)

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tool_condition

In [ ]:
def chatbot(state : MessagesState):
    result = model_with_tools.invoke(state["messages"])
    return {
        "messages" : [result]
    }

In [ ]:
graph = (
    StateGraph(MessagesState)
    .add_node("chatbot", chatbot)
    .add_node("tools",ToolNode(tools))

    .add_edge(START, "chatbot")

    .add_conditional_edges("chatbot",tools_condition)
    .add_edge("tools","chatbot")
    .compile()
)

In [ ]:
diagram = Image(
    graph.get_graph().draw_mermaid_png()
)
display(diagram)

In [ ]:
out = graph.invoke(
    {
        "messages" : "문벼리의 전화번호를 알려줘"
    }
)

In [ ]:
for msg in out['messages']:
    msg.pretty_print()

## With Memory

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
graph_memory = (
    StateGraph(MessagesState)
    .add_node("chatbot", chatbot)
    .add_node("tools", ToolNode(tools))

    .add_edge(START, "chatbot")

    .add_conditional_edges("chatbot", tool_condition)
    .add_edge("tools","chatbot")
    .compile(checkpointer=memory)
) 

In [ ]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    configurable={"thread_id":"User-A"},
)

In [ ]:
out=graph_memory.invoke(
    {
        "messages": "문벼리의 전화번호를 알려주세요"
    },
    config
)

In [ ]:
for msg in out['messages']:
    msg.pretty_print

In [ ]:
config = RunnableConfig(
    configurable={"thread_id":"User-B"},
)

In [ ]:
out=graph_memory.invoke(
    {
        "messages": "방금 찾은 전화번호에 국가코드 1을 더해라"
    },
    config
)

In [ ]:
for msg in out['messages']:
    msg.pretty_print